face_recognition_project/
│
├── data/                   # Directorio para datos
│   ├── raw/                # Imágenes crudas capturadas de la cámara
│   ├── processed/          # Imágenes preprocesadas (escaladas, recortadas, etc.)
│   └── embeddings/         # Representaciones numéricas generadas por el modelo
│
├── models/                 # Directorio para modelos entrenados y checkpoints
│   ├── face_detector/      # Modelo para detección de rostros
│   └── face_classifier/    # Modelo para clasificación de rostros
│
├── scripts/                # Scripts principales
│   ├── train.py            # Script para entrenar el modelo
│   ├── detect_faces.py     # Script para detección de rostros
│   └── classify_faces.py   # Script para clasificación de rostros
│
├── utils/                  # Funciones auxiliares
│   ├── preprocessing.py    # Funciones para procesar imágenes
│   ├── data_loader.py      # Funciones para cargar datos
│   └── metrics.py          # Evaluación de precisión, velocidad, etc.
│
├── tests/                  # Pruebas para verificar la funcionalidad
│   ├── test_detection.py   # Pruebas para el modelo de detección
│   └── test_classification.py # Pruebas para el modelo de clasificación
│
├── requirements.txt        # Dependencias del proyecto
├── README.md               # Descripción del proyecto
└── main.py                 # Script principal para ejecutar el sistema


In [1]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import joblib
from deepface import DeepFace  # Puedes usar cualquier librería compatible

# Ruta al modelo preentrenado
MODEL_PATH = "models/face_classifier/svm_model.pkl"

def load_model():
    """Carga el modelo preentrenado."""
    return joblib.load(MODEL_PATH)

def detect_faces(frame):
    """Detecta rostros en la imagen usando Haar Cascades."""
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

def classify_face(face_img, model, label_encoder):
    """Clasifica el rostro detectado."""
    embedding = DeepFace.represent(face_img, model_name="Facenet")[0]["embedding"]
    prediction = model.predict([embedding])
    return label_encoder.inverse_transform(prediction)[0]

def main():
    # Cargar modelo y etiquetas
    model = load_model()
    label_encoder = LabelEncoder()
    label_encoder.classes_ = np.load("models/face_classifier/labels.npy", allow_pickle=True)
    
    # Iniciar la cámara
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detección de rostros
        faces = detect_faces(frame)
        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            
            # Clasificación del rostro
            try:
                name = classify_face(face_img, model, label_encoder)
            except:
                name = "Desconocido"
            
            # Dibujar el cuadro y el nombre
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
        # Mostrar el video en vivo
        cv2.imshow("Reconocimiento Facial", frame)

        # Salir con 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()



25-01-13 11:34:04 - Directory C:\Users\nel_n\.deepface has been created
25-01-13 11:34:04 - Directory C:\Users\nel_n\.deepface\weights has been created


FileNotFoundError: [Errno 2] No such file or directory: 'models/face_classifier/svm_model.pkl'

Próximos Pasos 
Entrena el modelo de clasificación:
Usa un dataset de imágenes de tu compañero y tú.
Genera embeddings con una librería como DeepFace y entrena un clasificador (SVM/KNN).


Optimiza el rendimiento:
· Preprocesa las imágenes (escalado, normalización).
· Usa modelos ligeros para la detección (por ejemplo, Mediapipe).

Prueba y depura:
Asegúrate de que el sistema funcione en diferentes condiciones de iluminación.
Agrega pruebas unitarias para cada componente.
